In [114]:
from sklearn.datasets import load_breast_cancer
import numpy as np
import random

data = load_breast_cancer()
data.target[[10, 50, 85]]

list(data.target_names)


['malignant', 'benign']

In [115]:
X = data.data
y = data.target
full_data = np.concatenate([X, y.reshape(-1, 1)], axis=1)

header = list(data.feature_names) + ['labels']

In [116]:
test_data = full_data[400:]
training_data = full_data[:400]

In [117]:
def unique_vals(rows, col):
    return set([row[col] for row in rows])

In [118]:
def class_counts(rows):
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label in counts:
            counts[label] += 1
        else:
            counts[label] = 1
    return counts

In [119]:
class_counts(training_data)

{0.0: 173, 1.0: 227}

In [120]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [121]:
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [122]:
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    
    return true_rows, false_rows

In [123]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    
    for label in counts:
        prob = counts[label] / len(rows)
        impurity -= prob**2
        
    return impurity

$G = 1 - \sum\limits_k (p_k)^2$

In [124]:
some_mixing = [[0],
               [1], 
               [1]]
               
gini(some_mixing)

0.4444444444444444

In [125]:
def info_gain(left, right, current_uncertainty):
    info_gain = current_uncertainty
  
    left_uncertainty = gini(left)
    right_uncertainty = gini(right)
    
    info_gain -= len(left)/(len(left) + len(right)) * left_uncertainty
    info_gain -= len(right)/(len(left) + len(right)) * right_uncertainty
    
    return info_gain

$\Large IG(Q) = S_O - \sum_{i=1}^{q}\frac{N_i}{N}S_i$

In [126]:
def find_best_split(rows):
    best_gain = 0  
    best_question = None  
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  

    for col in range(n_features):  
        values = set([row[col] for row in rows]) 
        
        for val in values:  
            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)

            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            gain = info_gain(false_rows, true_rows, current_uncertainty)

            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [127]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [128]:
class Decision_Node:
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [129]:
def build_tree(rows):
    gain, question = find_best_split(rows)

    if gain == 0:
        return Leaf(rows)

    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)

    return Decision_Node(question, true_branch, false_branch)

In [130]:
def classify(row, node):

    if isinstance(node, Leaf):
        return node.predictions

    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [131]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [132]:
def predict(test, my_tree):
  result = []
  for row in test:
    result.append(list(print_leaf(classify(row, my_tree)).keys())[0])

  return result

In [133]:
def bagging(data, test, k=2):
  pred = []
  first = np.where(data[:,-1] == 1)[0]
  second = np.where(data[:,-1] == 0)[0]

  for i in range(k):
    low_f = random.randint(0, first.shape[0] - 100)
    high_f = random.randint(100, first.shape[0])

    low_s = random.randint(0, second.shape[0] - 100)
    high_s = random.randint(100, second.shape[0])

    split_data = np.concatenate([data[low_f:high_f], data[low_s:high_s]], axis=0)
    my_tree = build_tree(split_data)

    pred.append(predict(test, my_tree))

  return np.mean(pred, axis=0) 



In [134]:
res = bagging(training_data, test_data)

In [135]:
from sklearn.metrics import roc_auc_score

In [136]:
roc_auc_score(test_data[:, -1], res)

0.9101577909270216

In [137]:
my_tree = build_tree(training_data)

In [138]:
pred_tree = predict(test_data, my_tree)

In [139]:
roc_auc_score(test_data[:, -1], pred_tree)

0.9153846153846154

In [140]:
def print_tree(node, spacing=""):


    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    print (spacing + str(node.question))

    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [141]:
print_tree(my_tree)

Is worst perimeter >= 105.3?
--> True:
  Is worst concavity >= 0.221?
  --> True:
    Is mean smoothness >= 0.08401?
    --> True:
      Is mean texture >= 16.67?
      --> True:
        Predict {0.0: 138}
      --> False:
        Is worst area >= 812.4?
        --> True:
          Is mean concave points >= 0.05069?
          --> True:
            Predict {0.0: 12}
          --> False:
            Predict {1.0: 1}
        --> False:
          Predict {1.0: 2}
    --> False:
      Is worst fractal dimension >= 0.07474?
      --> True:
        Predict {1.0: 4}
      --> False:
        Predict {0.0: 5}
  --> False:
    Is worst area >= 1150.0?
    --> True:
      Predict {0.0: 4}
    --> False:
      Predict {1.0: 9}
--> False:
  Is worst smoothness >= 0.1785?
  --> True:
    Predict {0.0: 7}
  --> False:
    Is worst fractal dimension >= 0.05521?
    --> True:
      Is worst symmetry >= 0.5774?
      --> True:
        Predict {0.0: 1}
      --> False:
        Is worst area >= 750.1?
    